In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [10]:
num = input ("Input:") 
browser = webdriver.Chrome()
browser.get("http://image-net.org/")
inputElems = browser.find_elements_by_id('searchbox')
for inputElem in inputElems:
    inputElem.send_keys(num)
    inputElem.send_keys(Keys.ENTER)
    
result = browser.find_element_by_xpath("//span[@class='result_synset']")
result.click()
browser.implicitly_wait(10)
download = browser.find_element_by_xpath("//a[@href='#downloads']")
download.click()
download_link = browser.find_element_by_link_text('URLs')
download_link.click()

urls = browser.find_element_by_xpath("//pre").text.split()


Input:cat


In [13]:
import numpy as np
import urllib
import cv2

# Function to convert an URL to image
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read it into OpenCV format
    try:
        resp = urllib.request.urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    except:
        image = None
    return image

# Function to save image files to the current working directory, given an array of image URLS
def save_images(urls):
    for url in urls:
        image = url_to_image(url)
        if image is None:
            continue
        # Use string after last '/' in URL as the filename for the image
        filename = url.rsplit("/",1)[1]
        cv2.imwrite(filename, image)

In [ ]:
import os

base_dir = os.getcwd()

# Create directories for the images if they don't already exist
for directory in ("train/", "test/"):
    if not os.path.exists(directory + num):
        os.makedirs(directory + num)

# Get index to split training and test data (80% & 20%)
data_split_index = round(len(urls) * 0.8)

# Save training images to corresponding directory
os.chdir("train/" + num)
save_images(urls[0:data_split_index])

# Save test images to corresponding directory
os.chdir(base_dir + "/test/" + num)
save_images(urls[data_split_index:])

# Change working directory back to the base directory
os.chdir(base_dir)